In [ ]:
%pip install datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset("paws", "labeled_final")

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import Trainer, TrainingArguments
def preprocess_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
tokenized_dataset = dataset.map(preprocess_function, batched=True)
X_train = tokenized_dataset["train"]
X_test = tokenized_dataset["test"]
print(f"Sample tokenized input IDs: {X_train[0]['input_ids']}")
print(f"Sample tokenized attention mask: {X_train[0]['attention_mask']}")

In [6]:
X_valid=tokenized_dataset["validation"]

In [ ]:
%pip install transformers[torch]

In [ ]:
import os
from transformers import AdamW, get_scheduler, TrainingArguments, Trainer, AutoConfig
os.environ["WANDB_DISABLED"] = "true"

# Custom optimizer with regularization (weight decay)
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in ["bias", "LayerNorm.weight"])],
        'weight_decay': 0.01  # Apply L2 regularization to these parameters
    },
    {
        'params': [p for n, p in model.named_parameters() if any(nd in n for nd in ["bias", "LayerNorm.weight"])],
        'weight_decay': 0.0  # No regularization for bias and LayerNorm weights
    }
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

num_training_steps = len(X_train) // 8 * 4
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_training_steps
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir=None,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=None
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_valid,
    optimizers=(optimizer, lr_scheduler)
)

trainer.train()

# Evaluate on the test set
eval_results = trainer.evaluate(X_test)
print(f"Test Set Results: {eval_results}")


In [ ]:
model_save_path = './saved_model'
model.save_pretrained(model_save_path)  # Save the trained model